In [1]:
import nltk
import datetime as datetime 
import pickle 
import pandas as pd 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
mw_dataset_no_dup_no_troll = pd.read_csv('fb comments omicron only with correct date SG ONLY removed trolls after removing name tags.csv', encoding='latin1')

In [3]:
len(mw_dataset_no_dup_no_troll)

64234

In [4]:
mw_dataset_no_dup_no_troll.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,page_id,post_id,user_id,username,entry_id,entry_text,entry_timestamp,creation_time,anger_score,fear_score,joy_score,sadness_score,valence,emotion_score,sentiment_score,emotion_category,sentiment_category,utc_time,sg_time,date,processed_content
0,0,0,2.0,175537.0,9.388943e+10,93889432933_10158650785047934,0.0,-,1.015865e+16,"2021 repeat 2020, wuhan delta. 2022 repeat 202...",0.0,1.637977e+09,0.386,0.380,0.326,0.379,0.474,4.0,-1.0,anger,negative,2021-11-27 01:42:37,2021-11-27 17:42:37,2021-11-27,2021 repeat 2020 wuhan delta 2022 repeat 2021 ...
1,1,1,24.0,163330.0,5.923086e+14,592308557475467_4963343103705302,0.0,-,4.963351e+15,why not call OMG,0.0,1.637925e+09,0.371,0.336,0.182,0.432,0.343,4.0,-1.0,sadness,negative,2021-11-26 11:03:09,2021-11-27 03:03:09,2021-11-27,why not call OMG
2,2,2,37.0,163331.0,5.923086e+14,592308557475467_4963343103705302,0.0,-,4.963365e+15,The world had enough...ð,0.0,1.637925e+09,0.460,0.484,0.184,0.604,0.314,6.0,-1.0,sadness,negative,2021-11-26 11:09:27,2021-11-27 03:09:27,2021-11-27,The world had enoughð
3,3,3,40.0,163332.0,5.923086e+14,592308557475467_4963343103705302,0.0,-,4.963368e+15,Ooh aaah scary. I hear they have the LIGMA var...,0.0,1.637925e+09,0.408,0.545,0.319,0.448,0.422,5.0,-1.0,fear,negative,2021-11-26 11:10:41,2021-11-27 03:10:41,2021-11-27,Ooh aaah scary I hear they have the LIGMA vari...
4,4,4,45.0,163333.0,5.923086e+14,592308557475467_4963343103705302,0.0,-,4.963375e+15,"So how? Wait for it come, then u blur?",0.0,1.637925e+09,0.427,0.402,0.307,0.415,0.406,4.0,-1.0,anger,negative,2021-11-26 11:13:24,2021-11-27 03:13:24,2021-11-27,So how Wait for it come then u blur


In [5]:
mw_dataset_no_dup_no_troll.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1',
       'page_id', 'post_id', 'user_id', 'username', 'entry_id', 'entry_text',
       'entry_timestamp', 'creation_time', 'anger_score', 'fear_score',
       'joy_score', 'sadness_score', 'valence', 'emotion_score',
       'sentiment_score', 'emotion_category', 'sentiment_category', 'utc_time',
       'sg_time', 'date', 'processed_content'],
      dtype='object')

In [6]:
mw_dataset_no_dup_no_troll['processed_content'] = mw_dataset_no_dup_no_troll['processed_content'].astype(str)

In [7]:
import inflect
import re
import string
import unicodedata

def clean_text_1(headline):
    headline = headline.lower()
    headline = re.sub('\[.,*?\]','',headline)
    headline = re.sub('[0-9]','',headline)
    headline = re.sub('[%s]' % re.escape(string.punctuation), '', headline)
    headline = re.sub('[‘’“”…–]', '', headline)
    headline = re.sub('\w*\d\n\w*', '', headline)
    return headline

#def replace_contractions(text):
    #"""Replace contractions in string of text"""
    #return contractions.fix(text)

def remove_mentions(sample):
    """Remove mentions from a sample string"""
    return re.sub(r"@\S+", "", str(sample))
    
def remove_URL(sample):
    """Remove URLs from a sample string"""
    return re.sub(r"http\S+", "", str(sample))

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', str(word))
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words


def normalize(words):
    #words = replace_numbers(words)
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    return words

def preprocess(sample):
    sample = remove_URL(sample)
    sample= remove_mentions(sample)
    #sample = replace_contractions(sample)
    # Tokenize
    words = nltk.word_tokenize(sample)

    # Normalize
    return normalize(words)


round_1 = lambda x: preprocess(x)

In [8]:
mw_dataset_no_dup_no_troll['processed_content'] = mw_dataset_no_dup_no_troll['processed_content'].apply(round_1)

In [9]:
mw_dataset_no_dup_no_troll['processed_content']=mw_dataset_no_dup_no_troll['processed_content'].apply(' '.join)

In [10]:
mw_dataset_no_dup_no_troll['processed_content'].head()

0    2021 repeat 2020 wuhan delta 2022 repeat 2021 ...
1                                     why not call omg
2                                 the world had enough
3    ooh aaah scary i hear they have the ligma vari...
4                  so how wait for it come then u blur
Name: processed_content, dtype: object

In [11]:
mw_dataset_no_dup_no_troll.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,page_id,post_id,user_id,username,entry_id,entry_text,entry_timestamp,creation_time,anger_score,fear_score,joy_score,sadness_score,valence,emotion_score,sentiment_score,emotion_category,sentiment_category,utc_time,sg_time,date,processed_content
0,0,0,2.0,175537.0,9.388943e+10,93889432933_10158650785047934,0.0,-,1.015865e+16,"2021 repeat 2020, wuhan delta. 2022 repeat 202...",0.0,1.637977e+09,0.386,0.380,0.326,0.379,0.474,4.0,-1.0,anger,negative,2021-11-27 01:42:37,2021-11-27 17:42:37,2021-11-27,2021 repeat 2020 wuhan delta 2022 repeat 2021 ...
1,1,1,24.0,163330.0,5.923086e+14,592308557475467_4963343103705302,0.0,-,4.963351e+15,why not call OMG,0.0,1.637925e+09,0.371,0.336,0.182,0.432,0.343,4.0,-1.0,sadness,negative,2021-11-26 11:03:09,2021-11-27 03:03:09,2021-11-27,why not call omg
2,2,2,37.0,163331.0,5.923086e+14,592308557475467_4963343103705302,0.0,-,4.963365e+15,The world had enough...ð,0.0,1.637925e+09,0.460,0.484,0.184,0.604,0.314,6.0,-1.0,sadness,negative,2021-11-26 11:09:27,2021-11-27 03:09:27,2021-11-27,the world had enough
3,3,3,40.0,163332.0,5.923086e+14,592308557475467_4963343103705302,0.0,-,4.963368e+15,Ooh aaah scary. I hear they have the LIGMA var...,0.0,1.637925e+09,0.408,0.545,0.319,0.448,0.422,5.0,-1.0,fear,negative,2021-11-26 11:10:41,2021-11-27 03:10:41,2021-11-27,ooh aaah scary i hear they have the ligma vari...
4,4,4,45.0,163333.0,5.923086e+14,592308557475467_4963343103705302,0.0,-,4.963375e+15,"So how? Wait for it come, then u blur?",0.0,1.637925e+09,0.427,0.402,0.307,0.415,0.406,4.0,-1.0,anger,negative,2021-11-26 11:13:24,2021-11-27 03:13:24,2021-11-27,so how wait for it come then u blur


In [12]:
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

stemmer = SnowballStemmer("english")

mw_dataset_no_dup_no_troll['lemmatized_content'] = mw_dataset_no_dup_no_troll['processed_content'].astype('str').apply(lambda x: filter(None,x.split(" ")))
mw_dataset_no_dup_no_troll['lemmatized_content'] = mw_dataset_no_dup_no_troll['lemmatized_content'].apply(lambda x : [stemmer.stem(y) for y in x])
mw_dataset_no_dup_no_troll['lemmatized_content'] = mw_dataset_no_dup_no_troll['lemmatized_content'].apply(lambda x : " ".join(x))

In [13]:
mw_dataset_no_dup_no_troll.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,page_id,post_id,user_id,username,entry_id,entry_text,entry_timestamp,creation_time,anger_score,fear_score,joy_score,sadness_score,valence,emotion_score,sentiment_score,emotion_category,sentiment_category,utc_time,sg_time,date,processed_content,lemmatized_content
0,0,0,2.0,175537.0,9.388943e+10,93889432933_10158650785047934,0.0,-,1.015865e+16,"2021 repeat 2020, wuhan delta. 2022 repeat 202...",0.0,1.637977e+09,0.386,0.380,0.326,0.379,0.474,4.0,-1.0,anger,negative,2021-11-27 01:42:37,2021-11-27 17:42:37,2021-11-27,2021 repeat 2020 wuhan delta 2022 repeat 2021 ...,2021 repeat 2020 wuhan delta 2022 repeat 2021 ...
1,1,1,24.0,163330.0,5.923086e+14,592308557475467_4963343103705302,0.0,-,4.963351e+15,why not call OMG,0.0,1.637925e+09,0.371,0.336,0.182,0.432,0.343,4.0,-1.0,sadness,negative,2021-11-26 11:03:09,2021-11-27 03:03:09,2021-11-27,why not call omg,whi not call omg
2,2,2,37.0,163331.0,5.923086e+14,592308557475467_4963343103705302,0.0,-,4.963365e+15,The world had enough...ð,0.0,1.637925e+09,0.460,0.484,0.184,0.604,0.314,6.0,-1.0,sadness,negative,2021-11-26 11:09:27,2021-11-27 03:09:27,2021-11-27,the world had enough,the world had enough
3,3,3,40.0,163332.0,5.923086e+14,592308557475467_4963343103705302,0.0,-,4.963368e+15,Ooh aaah scary. I hear they have the LIGMA var...,0.0,1.637925e+09,0.408,0.545,0.319,0.448,0.422,5.0,-1.0,fear,negative,2021-11-26 11:10:41,2021-11-27 03:10:41,2021-11-27,ooh aaah scary i hear they have the ligma vari...,ooh aaah scari i hear they have the ligma vari...
4,4,4,45.0,163333.0,5.923086e+14,592308557475467_4963343103705302,0.0,-,4.963375e+15,"So how? Wait for it come, then u blur?",0.0,1.637925e+09,0.427,0.402,0.307,0.415,0.406,4.0,-1.0,anger,negative,2021-11-26 11:13:24,2021-11-27 03:13:24,2021-11-27,so how wait for it come then u blur,so how wait for it come then u blur


In [14]:
mw_dataset_no_dup_no_troll.to_csv('processed omicron fb dataset final.csv')

In [15]:
len(mw_dataset_no_dup_no_troll)

64234